# 系統維護工具

V0.1.1

# Init All

In [1]:
import pandas as pd
from codes.db import * 
from codes.lib import *
from codes.riverlog import *
import pandasql as ps
import os

gd={}
conn=connect_db()
if 1:
    load_ods(gd)
    riverlog_info_setup(gd)

Connected to the PostgreSQL database...
sys 的 sheets:
 dict_keys(['description', 'table_def', 's_syspar', 's_info_line', 's_info_area', 's_timeseq_rpt', 's_repeat_job', 's_topology_kind', 's_gis_layer', 's_info_point', 's_topology_transfer', 's_topology_node', 's_topology_edge', 's_village_waterin', 's_waterin_qty', 's_waterwork_qty', 's_waterin_quality', 's_value_domain', 's_waterin_b']) 
basic 的 sheets:
 dict_keys(['table_def', 'b_表單說明', 'b_colmeta', 'b_水資源分區', 'b_水資源局', 'b_河川局', 'b_流域', 'b_河川', 'b_排水', 'b_水庫', 'b_水質水量保護區', 'b_水庫集水範圍', 'b_水庫集水區敏感區', 'b_堤防', 'b_排水設施', 'b_水門', 'b_抽水站', 'b_河川斷面樁', 'b_雨量站', 'b_河川水位站', 'b_浮標站', 'b_潮位站', 'b_地下水分區', 'b_地下水觀測井']) 
rain-station: output/rain-station.csv saved, shape = (2149, 6)
reservoir-info: output/reservoir-info.csv saved, shape = (152, 22)
waterLevel-station: output/waterLevel-station.csv saved, shape = (5176, 13)
waterLevelDrain-station: output/waterLevelDrain-station.csv saved, shape = (1189, 4)
waterLevelAgri-station: output/waterLevelA

# 維護

將以下的 ods 灌入 DB

sys.ods, basic.ods, riverlog 基本資料

In [2]:
import subprocess
import os
import re
def exec_print(cmd_str):
    print("executing : %s" %(cmd_str)) 
    p = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in p.stdout.readlines():
        print(line)
    retval = p.wait()    
    
def str_tran(line):
    ret = line
    g = re.match( r'(\d*)年(\d*)月(\d*)日', line, re.M|re.I)

    if g:
        year = int(g.group(1))+1911
        date_str = "%i-%s-%s" %(year,g.group(2),g.group(3))
        #print(date_str)
        ret=date_str
    return ret

def ods_to_scr(tables,sql_name,psql_name,sql_drop_name, white_list=None):
    """
    white_list: 如果有，就只處理這些表
    """
    loc="/Volumes/F2020/opt/anaconda3/envs/py37/bin/csvsql"
    sqls=[]
    sqls_drop=[]
    table_list=[]
    if 'table_def' in tables:
        def_df=tables['table_def']
        table_list = def_df[def_df['type']=='T']['table_name'].to_list()
        print("table_list = %s" %(table_list))
    for t_name in tables:
        if (not t_name=='table_def') and t_name in table_list:
            if white_list and not (t_name in white_list):
                print("table:%s PASS" %(t_name))
                pass
            else:
                df = tables[t_name]
                sqls_drop.append("DROP TABLE IF EXISTS %s;" %(t_name))

                filename = "output/%s.csv" %(t_name)
                if t_name=="s_waterwork_qty" or t_name=="s_waterin_qty":
                    df['date'] = df['date'].apply(str_tran)
                    df['date']= pd.to_datetime(df['date']).dt.date
                    df_to_db(t_name,df,'replace')
                elif t_name=='s_waterin_b': #use df_to_db list
                    df_to_db(t_name,df,'replace')
                    sql_acts=[]
                    sql_acts.append("SELECT AddGeometryColumn('s_waterin_b','geom','3826','POINT','2')")
                    sql_acts.append("UPDATE s_waterin_b SET geom = ST_PointFromText(wkt_geom)")
                    sql_acts.append("CREATE INDEX s_waterin_b_geom_idx ON public.s_waterin_b USING gist (geom)")
                    for sql in sql_acts:
                        sql_exec(conn,sql)
                        
                else: #using csvsql
                    df.to_csv(filename,index=False)
                    cmd_str="head -n 1000 output/%s.csv | %s -i postgresql --no-constraints --tables '%s' >> %s" %(t_name,loc,t_name,sql_name)
                    exec_print(cmd_str)
                    psql="\\copy \"%s\" FROM 'output/%s.csv' WITH (FORMAT csv,HEADER);" %(t_name,t_name)
                    sqls.append(psql)
    
    with open(psql_name, 'a') as f:
        for line in sqls:
            f.write("%s\n" %(line))

    with open(sql_drop_name, 'a') as f:
        for line in sqls_drop:
            f.write("%s\n" %(line))

sql_drop_name="output/tb_drop.sql"
sql_name = "output/tb.sql"
psql_name = "output/tb.psql"

#產生 sql, psql script
if 1:
    if os.path.exists(sql_drop_name):
        os.remove(sql_drop_name)
    if os.path.exists(sql_name):
        os.remove(sql_name)
    if os.path.exists(psql_name):
        os.remove(psql_name)

    #ods_to_scr(gd['sys'],sql_name,psql_name,sql_drop_name,['s_village_waterin','s_waterin_qty','s_waterwork_qty','s_waterin_quality'])
    ods_to_scr(gd['sys'],sql_name,psql_name,sql_drop_name,['s_waterin_b'])

    #ods_to_scr(gd['basic'],sql_name,psql_name,sql_drop_name)
    #ods_to_scr(gd['riverlog'],sql_name,psql_name,sql_drop_name)

#manual modify schema
# r_waterlevel_station,b_河川水位站
    
#drop, create table
if 0:
    conn=connect_db()
    with open (sql_drop_name, "r") as scrfile:
        sqls=scrfile.readlines()

    sql_str = "".join(sqls)
    print("sql_drop script:\n%s" % (sql_str))
    sql_exec(conn,sql_str)
    
    with open (sql_name, "r") as scrfile:
        sqls=scrfile.readlines()

    sql_str = "".join(sqls)
    print("sql script:\n%s" % (sql_str))
    sql_exec(conn,sql_str)

#import data
if 0:
    psql="/Applications/Postgres.app/Contents/Versions/13/bin/psql"
    with open (psql_name, "r") as sqlfile:
        sqls=sqlfile.readlines()
    psql_str = "".join(sqls)
    print("psql script:\n%s" %(psql_str))

    cmd_str="%s -h localhost -p 5431 -U postgres postgis -f %s -a" %(psql,psql_name)
    exec_print(cmd_str)



table_list = ['s_syspar', 's_table_desc', 's_info_point', 's_info_line', 's_info_area', 's_timeseq_rpt', 's_repeat_job', 's_topology_kind', 's_topology_transfer', 's_topology_node', 's_topology_edge', 's_village_waterin', 's_waterin_qty', 's_waterwork_qty', 's_waterin_quality', 's_value_domain', 's_waterin_b']
table:s_syspar PASS
table:s_info_line PASS
table:s_info_area PASS
table:s_timeseq_rpt PASS
table:s_repeat_job PASS
table:s_topology_kind PASS
table:s_info_point PASS
table:s_topology_transfer PASS
table:s_topology_node PASS
table:s_topology_edge PASS
table:s_village_waterin PASS
table:s_waterin_qty PASS
table:s_waterwork_qty PASS
table:s_waterin_quality PASS
table:s_value_domain PASS
PostgreSQL Table s_waterin_b has been created successfully.


/Volumes/F2020/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sqlalchemy/dialects/postgresql/base.py:3199: SAWarning: Did not recognize type 'geometry' of column 'geom'
  "Did not recognize type '%s' of column '%s'" % (attype, name)


# 使用範例

In [ ]:
#gd['base']['拓墣-Node']
#gd['riverlog']["r_rain_station"]
if 0:
    df = gd['basic']['河川局']
    #query_str = """SELECT * FROM df_8410 where site_id like '%東區%'"""
    query_str = """SELECT * FROM df where rvb_name='第一河川局'"""
    q1_df= ps.sqldf(query_str, locals())
#q1_df


# DB 存取範例

In [3]:
conn=connect_db()

if 0:
    sql = "select * from rivercode order by river_id"
    df = sql_to_df(conn,sql)

if 0:
    df = gd['basic']['河川局']
    df.to_csv('output/河川局.csv')    

if 1:
    sql="""
    CREATE TABLE "河川局" (
        a DECIMAL, 
        rvb_no DECIMAL, 
        rvb_name VARCHAR, 
        area BOOLEAN, 
        "水資源分區代號" BOOLEAN
    );

    """
    sql_exec(conn,sql)
if 0:
    sql = 'select * from 河川局'
    df = sql_to_df(conn,sql)
if 1:
    close_db(conn)
#df

Connecting to the PostgreSQL database...
Database connection closed.


## conn=connect_db()

if 0:
    sql = "select * from rivercode order by river_id"
    df = sql_to_df(conn,sql)

if 0:
    df = gd['basic']['河川局']
    df.to_csv('output/河川局.csv')    

if 1:
    sql="""
    CREATE TABLE "河川局" (
        a DECIMAL, 
        rvb_no DECIMAL, 
        rvb_name VARCHAR, 
        area BOOLEAN, 
        "水資源分區代號" BOOLEAN
    );

    """
    sql_exec(conn,sql)
if 0:
    sql = 'select * from 河川局'
    df = sql_to_df(conn,sql)
if 1:
    close_db(conn)
#df

# shp 匯入資料庫

用 data 目錄下的 shp 產生 script, 再手動到命令列執行，灌入資料庫

In [3]:
#pathnames_ori = walk_dir("./data",".shp",['工業區污水處理廠分布_121.shp']) #
pathnames_ori = walk_dir("./data",".SHP") #
pathnames = list_remove(pathnames_ori,['_Sanhe'])


#print(pathnames)
shp_tosql(pathnames,"","data/shp.sql",3826,"Big5")
#shp_tosql(pathnames,"8818-工業區污水處理廠分布位置圖","data/shp.sql",102443,"UTF-8")
#shp_tosql(lines,,table_name, sql_path="data/shp.sql", srid=3826, encoding="UTF-8"):

shp2pgsql -W Big5 -D -s 3826 -I "./data/新竹縣市人口數據/109年6月新竹縣統計區人口統計_最小統計區_SHP/109年6月新竹縣統計區人口統計_最小統計區.SHP" "109年6月新竹縣統計區人口統計_最小統計區">> data/shp.sql
shp2pgsql -W Big5 -D -s 3826 -I "./data/新竹縣市人口數據/109年6月新竹市統計區人口統計_最小統計區_SHP/109年6月新竹市統計區人口統計_最小統計區.SHP" "109年6月新竹市統計區人口統計_最小統計區">> data/shp.sql
shp2pgsql -W Big5 -D -s 3826 -I "./data/新竹縣市人口數據/109年6月新竹縣統計區人口指標_最小統計區_SHP/109年6月新竹縣統計區人口指標_最小統計區.SHP" "109年6月新竹縣統計區人口指標_最小統計區">> data/shp.sql
shp2pgsql -W Big5 -D -s 3826 -I "./data/新竹縣市人口數據/109年6月新竹市統計區人口指標_最小統計區_SHP/109年6月新竹市統計區人口指標_最小統計區.SHP" "109年6月新竹市統計區人口指標_最小統計區">> data/shp.sql
psql -p 5431 -U postgres -d postgis -f data/shp.sql


# 讀取單月淨水場水質資料

In [ ]:
import pandas as pd
filename="/Volumes/F2020/MakerBk2/QGIS/projects/hackathon/自來水水質資訊_202105.csv"

mv_str="。"
df_csv = pd.read_csv(filename,header=[0,1,2,3])

if True:
    values = []
    for index, row in df_csv.iterrows():
        line = str(row['淨水場資訊'][0])
        cols = line.split("(")
        print(cols[0].strip())


# 單月淨水場水質 CSV 分析

In [20]:
#列出頭前溪流域淨水場的 總溶解固體量(Total Dissolved Solids)
import pandas as pd
filename="/Volumes/F2020/MakerBk2/QGIS/projects/hackathon/自來水水質資訊_202105.csv"
#targets=["第一淨水場","第二淨水場","東興淨水場","寶山淨水廠","員崠淨水場","梅花淨水場","尖石淨水場","內灣淨水場","桃山淨水場","芎林淨水場"]
targets=['新竹第一淨水場','新竹第二淨水場','東興淨水場','寶山淨水場','員崠淨水場','梅花淨水場','尖石淨水場','內灣淨水場','桃山淨水場','芎林淨水場']
df_csv = pd.read_csv(filename,header=[0,1,2,3])
item_name='總溶解固體量(Total Dissolved Solids)'

if True:
    idvalue = {}
    for col_info in df_csv.columns:
        if col_info[0]==item_name:
            #print(col_info)
            info_str="%s-單位:%s, 飲用水水質標準:%s" %(col_info[0],col_info[3],col_info[2])
            print(info_str)
            break
    for index, row in df_csv.iterrows():
        factory=row['淨水場名稱'][0]
        if factory in targets:
            value = row[item_name][0]            
            #print("%s:%s" %(factory,value))
            idvalue[factory]=value
    
    items = sorted(idvalue.items(), key=lambda x:x[1])
    for item in items:
        print("%s:%s" %(item[0],item[1]))





總溶解固體量(Total Dissolved Solids)-單位:mg/L, 飲用水水質標準:500
尖石淨水場:84.8
梅花淨水場:123.0
內灣淨水場:167.0
桃山淨水場:170.0
東興淨水場:187.0
芎林淨水場:237.0
新竹第二淨水場:242.0
寶山淨水場:246.0
員崠淨水場:253.0
新竹第一淨水場:263.0


In [ ]:
# 算感測值跟標準的比例
df_result['test'] = pd.to_numeric(df_result['飲用水水質標準'])
df_result
if 0:
    f1=df_result['飲用水水質標準'].str.isnumeric()
    df3=df_result[f1]
    df4=df3['percent']=df3['值']/df3['飲用水水質標準']*100
    df4
#df_out=df_result['percent']=df_result['值']/df_result['飲用水水質標準']*100
#df_out.to_csv('output/water_quality_result.csv')

# 淨水場水質資料灌入資料庫

In [ ]:
#
def lines_to_file(lines,filename):
    fp = open(filename, "w")
    fp.write("\n".join(lines))
    fp.close()
 
def update_waterwork_quantity(src_pairs,sql_filename):
    """ 將列的幾個淨水場水質報告檔案，產生灌入資料庫的 sql
    每個月的第一筆 sql 是 delete 同資料日期的資料，如果改同月資料日期，記得手動刪除
    """
    import pandas as pd

    idx_colname=['區處別','系統代號','淨水場名稱','淨水場資訊']
    idxs={}
    values={}
    sqls=[]

    for [filename, date_str] in src_pairs:
        sql="DELETE FROM m_waterwork_quality WHERE \"日期\" = '%s';" %(date_str)
        sqls.append(sql)
        idxs['日期']=date_str
        df_csv = pd.read_csv(filename,header=[0,1,2,3])
        for index, row in df_csv.iterrows():
            cols = row.keys() #('區處別', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Unnamed: 0_level_3')

            for col in cols:
                #print("%s=%s" %(col[0],row[col]))

                if col[0] in idx_colname:
                    idxs[col[0]]=row[col]
                else:
                    values['項目']=col[0]
                    if col[0]=='水質合格否(Y/N)':
                        values['值']=row[col]
                        values['項次']=0
                        values['飲用水水質標準']=""
                        values['單位']=""
                    else:
                        values['值']=row[col]
                        values['項次']=col[1]
                        values['飲用水水質標準']=col[2]
                        values['單位']=col[3]
                    #print("idxs=%s\nvalues=%s" %(idxs,values))


                    values_str = "'%s','%s','%s','%s','%s','%s','%s','%s','%s','%s'" %(idxs['日期'], idxs['區處別'], idxs['系統代號'],idxs['淨水場名稱'],values['項目'],values['值'],values['單位'],values['飲用水水質標準'],values['項次'],idxs['淨水場資訊'])

                    sql="""INSERT INTO m_waterwork_quality("日期", "區處別","系統代號","淨水場名稱","項目","值","單位","飲用水水質標準","項次","淨水場資訊") VALUES (%s);""" %(values_str)
                    #print(sql)
                    sqls.append(sql)
    lines_to_file(sqls,sql_filename)
    return sqls
src_pairs=[["data/自來水水質資訊_202104.csv",'2021-04-08'], \
           ["data/自來水水質資訊_202105.csv",'2021-05-24']]
sql_filename="data/waterwork_quality.sql"
sqls=update_waterwork_quantity(src_pairs,sql_filename)
print("%s saved" %(sql_filename))

if 1: # update DB
    i=0
    for sql in sqls:
        if i % 100 ==0:
            print("%i sqls processed!" %(i))
        sql_exec(conn,sql)
        i=i+1
    print("total %i sqls processed!" %(i))




# 環保署 API
以下 API 測試過

/stat_p_123 重要河川水質概況

/wqx_p_01 河川水質監測資料

/dws_p_28 每月自來水水質監測資料

/wqx_p_12 河川水質季監測資料

/ems_s_01 環境保護許可管理系統(暨解除列管)對象基本資料

/stat_p_44 垃圾處理場(廠)統計

/stat_p_45 垃圾清理量資料

/stat_p_87 垃圾處理場(廠)座數

In [35]:
# 這裡可以將 API 資料下載下來，轉換成 df
api_key="615b2545-254c-4300-8e66-c64af62d4cf0" #請更新成自己環保署API key
limit=1000
limit_break=0 #>0 can limit total
offset=0
load_cnt=0
renew_url=True
    
if 0: #/stat_p_123 重要河川水質概況 -> e_river_state_q
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_123?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_123-%i.json"
    use_label=True
    pass
if 0: # /wqx_p_01 河川水質監測資料 -> e_river_q
    url_t = "https://data.epa.gov.tw/api/v1/wqx_p_01?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/wqx_p_01-%i.json"
    use_label=False
if 0: #/dws_p_28 每月自來水水質監測資料 -> e_waterwork_q
    url_t = "https://data.epa.gov.tw/api/v1/dws_p_28?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/dws_p_28-%i.json"
    use_label=False
if 0: #/wqx_p_12 河川水質季監測資料 #"total": 117041 -> e_river_season_q
    url_t = "https://data.epa.gov.tw/api/v1/wqx_p_12?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/wqx_p_12-%i.json"
    use_label=False
if 0: #/ems_s_01 環境保護許可管理系統(暨解除列管)對象基本資料 = 118447-環境保護許可管理系統(暨解除列管)對象基本資料     
    #"total": 322225 -> e_factory_base
    url_t = "https://data.epa.gov.tw/api/v1/ems_s_01?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/ems_s_01-%i.json"
    use_label=False
if 0: #/stat_p_44 垃圾處理場(廠)統計 "total": 220  Garbage disposal site, 不進 DB
    url_t="https://data.epa.gov.tw/api/v1/stat_p_44?api_key=%s"
    url= url_t % (api_key)
    renew_url=False
    filename_t= "output/stat_p_44-%i.json"
    use_label=False
if 0: #/stat_p_45 垃圾清理量資料 "total": 902 -> e_trash_stat_qty
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_45?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_45-%i.json"
    use_label=False     
if 1: # /stat_p_46 垃圾清理回收率指標資料 "total": 2194 -> e_trash_recycle
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_46?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_46-%i.json"
    use_label=False 
if 0: #/stat_p_87 垃圾處理場(廠)座數 "total": 575 -> e_garbage_disposal
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_87?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_87-%i.json"
    use_label=False 
    
    
    
while True:
    if renew_url==True:
        url= url_t % (offset,limit,api_key)
    filename  = filename_t %(offset)
    print("url=%s" %(url))
    url_get(filename, url,False)
    data = load_json(filename)

    if offset==0:
        labels={}
        d={}
        #print("labels:")
        for i in range(len(data['fields'])):
            #print("%s=%s" %(data['fields'][i]['id'],data['fields'][i]['info']['label']))
            labels[data['fields'][i]['id']]=data['fields'][i]['info']['label']
        print("labels=%s" %(labels))
        print("total=%i" %(data['total']))

    #print("values:")
    records_cnt = len(data['records'])
    for i in range(records_cnt):
        record_cur = data['records'][i]
        #print("%i: %s " %(i, record_cur))
        for key in record_cur.keys():
            if use_label:
                if not labels[key] in d.keys():
                    #print("key=%s" %(key))
                    d[labels[key]]=[]
                d[labels[key]].append(record_cur[key])
            else:
                if not key in d.keys():
                    #print("key=%s" %(key))
                    d[key]=[]
                d[key].append(record_cur[key])
        load_cnt += 1
    #load_cnt += records_cnt
    print("load_cnt=%i" %(load_cnt))
    if load_cnt >= data['total'] or (limit_break>0 and load_cnt >= limit_break):
        break
    offset += limit

df = pd.DataFrame.from_dict(d)
#df_to_db('table_name',df,'replace')
df

url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=0&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
labels={'Year': '年度', 'Month': '月份', 'County': '縣市別', 'Garbage_Recycled': '資源回收率', 'Food_Waste': '廚餘回收率', 'Bulk_Waste': '巨大垃圾回收再利用率', 'Municipal_Waste_Recycling_Rate': '垃圾回收率'}
total=2194
load_cnt=1000
url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=1000&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
load_cnt=2000
url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=2000&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
load_cnt=2194


,Year,Month,County,Garbage_Recycled,Food_Waste,Bulk_Waste,Municipal_Waste_Recycling_Rate
0,110,3,臺南市,60.64,6.60,1.13,68.37
1,110,3,連江縣,30.20,33.25,6.43,69.87
2,110,3,嘉義市,54.02,4.49,0.75,59.26
3,110,3,宜蘭縣,55.92,5.28,1.02,62.22
4,110,3,南投縣,48.49,3.99,0.98,53.46
...,...,...,...,...,...,...,...
2189,105,3,宜蘭縣,41.22%,16.83%,1.35%,59.4%
2190,105,3,高雄市,44.6%,10.4%,0.46%,55.45%
2191,105,3,臺南市,47.62%,11.23%,1.31%,60.16%
2192,105,3,臺中市,46.69%,5.28%,0.54%,52.51%


In [5]:
#上傳資料庫，並建立資料表
#df.dtypes
df_to_db('6932-自來水用水量',df,'replace')
#df.to_csv("output/debug.csv")

PostgreSQL Table 6932-自來水用水量 has been created successfully.


# 氣象局 API

/v1/rest/datastore/C-B0027-001 月平均-局屬地面測站資料


In [56]:
    
# 這裡可以將 API 資料下載下來，轉換成 df
api_key="CWB-5560CA59-1269-4359-87B2-CAFA83EDE4E0" #請更新成自己環保署API key
limit=10
limit_break=0 #>0 can limit total
offset=0
load_cnt=0
renew_url=True
    
if 0: #/v1/rest/datastore/C-B0027-001 月平均-局屬地面測站資料, 新竹站
    stationId=467571
    url_t = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/C-B0027-001?Authorization=%s&limit=%i&offset=%i&format=JSON&stationId=%i&weatherElement=stationPressure,temperature,relativeHumidity,cloudCover,sunshineDuration,precipitation,windSpeed&dataMonth="
    filename_t= "output/C-B0027-001-%i.json"
    url= url_t % (api_key,limit,offset,stationId)
    renew_url=False

if 1: #/v1/rest/datastore/C-B0027-001 月平均-局屬地面測站資料, 全測站
    stationId=467571
    url_t = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/C-B0027-001?Authorization=%s&format=JSON&weatherElement=stationPressure,temperature,relativeHumidity,cloudCover,sunshineDuration,precipitation,windSpeed&dataMonth="
    filename_t= "output/C-B0027-001-%i.json"
    url= url_t % (api_key)
    renew_url=False
    
#目前不支援 limit，多筆的用法，因為這個 API 氣象局不限制比數   
d={}
while True:
    if renew_url==True:
        url= url_t % (api_key,limit,offset)
    filename  = filename_t %(offset)
    print("url=%s" %(url))
    url_get(filename, url,True)
    data = load_json(filename)
    
    if offset==0:
        labels={}
        #print("labels:")
        ids=data['result']['fields']
        for i in range(len(ids)):
            labels[ids[i]['id']]=ids[i]['id']
        print("ids=%s" %(labels))
    
    location = data['records']['data']['surfaceObs']['location']
    for i in range(len(location)):
    
        station = location[i]['station']

        temperature = location[i]['stationObsStatistics']['temperature']
        monthly = temperature['monthly']

        head={}
        tail={}
        for month in monthly:
            #print(month)
            #head
            for key in station.keys():
                head[key]=station[key]
                if not key in d:
                    d[key]=[]
                d[key].append(station[key])
            #tail
            for key in month.keys():
                tail[key]=month[key]
                if not key in d:
                    d[key]=[]

                d[key].append(month[key])
            #print("tail=%s" %(tail))
        load_cnt += 1

        #load_cnt += records_cnt
    print("load_cnt=%i" %(load_cnt))
    if (limit_break>0 and load_cnt >= limit_break):
        break
    offset += limit
    
    break


df = pd.DataFrame.from_dict(d)
df


url=https://opendata.cwb.gov.tw/api/v1/rest/datastore/C-B0027-001?Authorization=CWB-5560CA59-1269-4359-87B2-CAFA83EDE4E0&format=JSON&weatherElement=stationPressure,temperature,relativeHumidity,cloudCover,sunshineDuration,precipitation,windSpeed&dataMonth=
ids={'stationID': 'stationID', 'stationName': 'stationName', 'stationNameEN': 'stationNameEN', 'stationAttribute': 'stationAttribute', 'startYear': 'startYear', 'endYear': 'endYear', 'dataMonth': 'dataMonth', 'mean': 'mean', 'maximum': 'maximum', 'minimum': 'minimum', 'maxGE30Days': 'maxGE30Days', 'meanGE25Days': 'meanGE25Days', 'minLE10Days': 'minLE10Days', 'total': 'total', 'accumulation': 'accumulation', 'ge01Days': 'ge01Days'}
load_cnt=26


,stationID,stationName,stationNameEN,stationAttribute,dataMonth,mean,maximum,minimum,maxGE30Days,meanGE25Days,minLE10Days
0,466880,板橋,BANQIAO,局屬有人氣象站,1,16.3,19.6,13.7,0.0,0.0,3.5
1,466880,板橋,BANQIAO,局屬有人氣象站,2,17.0,20.9,14.1,0.4,0.0,2.1
2,466880,板橋,BANQIAO,局屬有人氣象站,3,18.6,22.7,15.4,1.4,0.6,0.8
3,466880,板橋,BANQIAO,局屬有人氣象站,4,22.2,26.5,18.9,8.1,6.6,0.1
4,466880,板橋,BANQIAO,局屬有人氣象站,5,25.7,30.0,22.4,17.6,19.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
307,467770,梧棲,WUQI,局屬有人氣象站,8,28.9,31.9,26.4,28.4,30.8,0.0
308,467770,梧棲,WUQI,局屬有人氣象站,9,27.6,30.8,25.1,22.4,28.5,0.0
309,467770,梧棲,WUQI,局屬有人氣象站,10,24.7,28.0,22.2,5.2,13.3,0.0
310,467770,梧棲,WUQI,局屬有人氣象站,11,21.8,25.1,19.3,0.2,1.7,0.1


In [59]:
#df.to_csv("~/Downloads/weather_avg.csv")
len(df['stationName'].unique())

26

In [54]:
location = data['records']['data']['surfaceObs']['location']
location
len(location)

26

In [26]:
#ids=data['result']['fields']
location = data['records']['data']['surfaceObs']['location']
station = location[0]['station']
#station['stationID']

temperature = location[0]['stationObsStatistics']['temperature']
monthly = temperature['monthly']

for month in monthly:
    print(month)

{'dataMonth': '1', 'mean': '15.7', 'maximum': '19.1', 'minimum': '13.1', 'maxGE30Days': '0.0', 'meanGE25Days': '0.0', 'minLE10Days': '4.5'}
{'dataMonth': '2', 'mean': '16.0', 'maximum': '19.4', 'minimum': '13.4', 'maxGE30Days': '0.1', 'meanGE25Days': '0.0', 'minLE10Days': '3.1'}
{'dataMonth': '3', 'mean': '18.0', 'maximum': '21.6', 'minimum': '15.2', 'maxGE30Days': '0.3', 'meanGE25Days': '0.3', 'minLE10Days': '0.8'}
{'dataMonth': '4', 'mean': '21.9', 'maximum': '25.6', 'minimum': '18.9', 'maxGE30Days': '2.6', 'meanGE25Days': '5.0', 'minLE10Days': '0.1'}
{'dataMonth': '5', 'mean': '25.2', 'maximum': '28.9', 'minimum': '22.2', 'maxGE30Days': '12.3', 'meanGE25Days': '17.8', 'minLE10Days': '0.0'}
{'dataMonth': '6', 'mean': '27.9', 'maximum': '31.5', 'minimum': '24.9', 'maxGE30Days': '24.0', 'meanGE25Days': '27.5', 'minLE10Days': '0.0'}
{'dataMonth': '7', 'mean': '29.3', 'maximum': '33.2', 'minimum': '26.0', 'maxGE30Days': '30.3', 'meanGE25Days': '31.0', 'minLE10Days': '0.0'}
{'dataMonth': 

# EPA CWMS 水量水質自動監測連線處理

In [4]:
filename="/Users/wuulong/Downloads/cwms.xml"
if 0: #新竹縣，正常
    url="HTTP://hsinchuauto.tk/cwmsopendata/cwms.xml"
    url_get(filename,url,reload=True)
    df=xml_to_df(filename)
    df_to_db('e_cwms_hsinchu',df)
if 0: # 新竹市，網站的 xml 檔案無法存取，無法繼續
    url="HTTP://cwms.hccepb.gov.tw/cwmsopendata/cwms.xml"
    url_get(filename,url,reload=True)
    df=xml_to_df(filename)
    df_to_db('e_cwms_hsinchucity',df)

PostgreSQL Table e_cwms_hsinchu has been created successfully.


In [10]:
#測試
mdate=df['M_DATE'].unique()[0]
sql="delete from e_cwms_hsinchu where \"M_DATE\"='%s'" %(mdate)
print(sql)

delete from e_cwms_hsinchu where "M_DATE"='1100704'


# 匯入 CSV

# Data Transfer Kit

In [4]:
filename = "data/6932-自來水用水量.csv"
df = pd.read_csv(filename)
df

,County,Month,TheDailyDomesticConsumptionOfWaterPerPerson,Town,Year
0,金門縣,1,31.0,金城鎮,104
1,金門縣,1,46.0,金寧鄉,104
2,金門縣,1,262.0,金湖鎮,104
3,金門縣,1,166.0,金沙鎮,104
4,金門縣,1,84.0,烈嶼鄉,104
...,...,...,...,...,...
25187,連江縣,2,NaN,南竿鄉,110
25188,連江縣,2,NaN,北竿鄉,110
25189,連江縣,2,NaN,西莒,110
25190,連江縣,2,NaN,東莒,110


In [34]:
f = open("/tmp/lines.txt")
lines = f.readlines()
f.close()

for line in lines:
    line=line.strip()
    #print("SELECT UpdateGeometrySRID(''%s','geom',3826);" %(line))
    print("select * from geometry_columns where f_table_name='%s';\n" %(line.lower()))
    


select * from geometry_columns where f_table_name='ppobsta_wra_e';

select * from geometry_columns where f_table_name='riverl';

select * from geometry_columns where f_table_name='rivwlsta_e';

select * from geometry_columns where f_table_name='rivqasta_e';

select * from geometry_columns where f_table_name='rivsesta';

select * from geometry_columns where f_table_name='hyctide';

select * from geometry_columns where f_table_name='hycw';

select * from geometry_columns where f_table_name='hycbuoy';

select * from geometry_columns where f_table_name='gwobwell_e';

select * from geometry_columns where f_table_name='lasubsta';

select * from geometry_columns where f_table_name='gwqmosta';

select * from geometry_columns where f_table_name='subasta';



# Debug

In [63]:
import datetime
print(datetime.datetime.now())

2021-07-08 12:35:03.459122


In [11]:
def str_tran(line):
    import re
    g = re.match( r'(\d*)年(\d*)月(\d*)日', line, re.M|re.I)

    if g:
        year = int(g.group(1))+1911
        date_str = "%i-%s-%s" %(year,g.group(2),g.group(3))
        #print(date_str)
        line=date_str
    return line

df=pd.read_csv("~/Downloads/s_waterwork_qty.csv")

df['date'] = df['date'].apply(str_tran)
df['date']= pd.to_datetime(df['date']).dt.date

df_to_db('t_test',df,'replace')

PostgreSQL Table t_test has been created successfully.
